In [2]:
!pip install google-cloud-speech


In [3]:
!pip install pytube openai-whisper
!pip install youtube_transcript_api
!pip install nltk
!pip install transformers
!pip install sumy
!pip install translate
!pip install googletrans==4.0.0-rc1

from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled, NoTranscriptAvailable, NoTranscriptFound, VideoUnavailable
from googletrans import Translator, LANGUAGES
import whisper
from pytube import YouTube
import warnings
from pytube.captions import Caption
import nltk
nltk.download('punkt')
import os
from transformers import pipeline
from nltk.tokenize import sent_tokenize
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers.lsa import LsaSummarizer
from transformers import pipeline, AutoTokenizer
import unicodedata
import logging
from translate import Translator

logging.getLogger("transformers").setLevel(logging.ERROR)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
from IPython.display import display

def eng_aliases():

    LANGUAGES['en-us'] = 'English'
    LANGUAGES['en-US'] = 'English'
    LANGUAGES['en-gb'] = 'English'
    LANGUAGES['en-GB'] = 'English'

def fetch_transcript_with_existing_subtitles(youtube_url):

    yt = YouTube(youtube_url)
    chars_to_remove = {':', ',', ';', '|', '.'}
    title = yt.title.translate(str.maketrans('', '', ''.join(chars_to_remove)))
    print("Video Title:", title)
    eng_aliases()


    available_captions = yt.captions.all()

    manual_caption = None
    for caption in available_captions:
        if caption.code in ["en", "en-us", "en-gb"]:
            manual_caption = caption
            break

    if manual_caption:
        manual_caption.download(title)

        with open(title + '.srt', 'r', encoding='utf-8') as file:
            subtitles = Caption.get_srt_captions(file.read())

        transcript = ' '.join([subtitle.text for subtitle in subtitles])
        return transcript
    else:
        return None

def fetch_transcript_with_ASR(youtube_url, has_audio_file=False):

    output_file = "output.web3"

    temp_dir = "temp"
    os.makedirs(temp_dir, exist_ok=True)
    yt = YouTube(youtube_url)
    title = yt.title

    name = title + ".mp4"
    file_location = "/content/" + temp_dir + "/" + name
    stream = yt.streams.filter(only_audio=True, file_extension="mp4").first()
    stream.download(output_path=temp_dir)
    model = whisper.load_model("base")
    result = model.transcribe(file_location)

    display(result["text"])
    return result["text"]

def get_summary(text, model_choice):
    if (model_choice == 0):
        extractive_summary = get_extractive_summary(text)
        return extractive_summary
    else:
        abstractive_summary = get_abstractive_summary(extractive_summary, model_choice)
        return abstractive_summary



def get_abstractive_summary(extractive_summary, model_choice):
    models = {
        0: "facebook/bart-large-cnn",
        1: "t5-base"
    }
    model_name = models[model_choice]

    tokenizer = AutoTokenizer.from_pretrained(model_name, model_max_length=1024)  # Adjust max length as needed

    # Load the model for summarization
    generator = pipeline('summarization', model=model_name)

    # Generate abstractive summary
    abstractive_summary = generator(extractive_summary, max_length=100, min_length=5, do_sample=True, early_stopping=True)[0]['summary_text']

    return abstractive_summary

def get_extractive_summary(text):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    req_sentences = round(len(sent_tokenize(text)) * 0.70)
    summarizer = TextRankSummarizer()
    summary = summarizer(parser.document, req_sentences)

    ext_summary = " ".join(str(sentence) for sentence in summary)
    return ext_summary

def clean_summary(text):
    error_dict = [...]
    for error in error_dict:
        if error == text:
            return text

    irrelevant_terms = ["[music]", "[Music]", "\n", "<<", ">>"]
    sentence_list = [sentence.replace(item, "").strip() for sentence in sent_tokenize(text) for item in irrelevant_terms]
    cleaned_text = " ".join(sentence.capitalize() for sentence in sentence_list)
    normalized_text = unicodedata.normalize('NFKD', cleaned_text)
    formatted_text = normalized_text.encode('ascii', 'ignore').decode('ascii')
    return formatted_text.replace("\'", "'")

def translate(text="type", source="source language", dest="destination language"):
    text1 = text
    source1 = source
    dest1 = dest
    trans = Translator()
    try:
        trans1 = trans.translate(text1, src=source1, dest=dest1)
        return trans1.text
    except Exception as e:
        print(f"Translation error: {e}")
        return None

def translate_summary(summary, source_lang, dest_lang):
    if not summary:
        print("Cannot translate empty summary.")
        return None

    try:
        translated_summary = translate(summary, source_lang, dest_lang)
        return translated_summary
    except Exception as e:
        print(f"Translation error: {e}")
        return None


def display_lang():
    print(list(LANGUAGES.values()))

if __name__ == "__main__":
    youtube_url = input("Enter the YouTube video URL: ")  # Change to your desired YouTube URL
    video_transcript = fetch_transcript_with_existing_subtitles(youtube_url)

    if video_transcript:
        print("Using Existing Subtitles:")
        display(video_transcript)
    else:
        print("No Existing Subtitles. Using ASR:")
        video_transcript = fetch_transcript_with_ASR(youtube_url)

    # Summarization
    model_choice = int(input("Choose a summarization model (0 for 'Extractive summary', 1 for 'Abstractive summary'): "))
    summary = get_summary(video_transcript, model_choice)
    print("Summary:")
    display(summary)

    # Translation
    lang_choice = input("Do you want to translate the summary? (yes/no): ").lower()
    if lang_choice == "yes":
        display_lang()
        source_lang = input("Enter the source language code: ")
        dest_lang = input("Enter the destination language code: ")
        translated_summary = translate(summary, source_lang, dest_lang)
        if translated_summary:
            print("Translated Summary:")
            display(translated_summary)



Enter the YouTube video URL: https://www.youtube.com/watch?v=lOA8rriYFB8&t=4s
Video Title: Why You SHOULD NOT Take Harvard CS50!
No Existing Subtitles. Using ASR:


<ipython-input-4-028e068f8add>:19: DeprecationWarning: Call to deprecated function all (This object can be treated as a dictionary).
  available_captions = yt.captions.all()
/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


" If you search YouTube for how to learn how to code today, one course comes up over and over again. The course is called CS15 by Harvard University. Harvard CS50 Introduction to Computer Science. This is CS50. So everyone loves it and gives it great recommendations. I don't think that Harvard CS50 is necessarily the right course for everyone. And in this video, I'm going to tell you why. If you don't know what Harvard CS50 is, essentially, it's a introductory computer science course offered at Harvard University that is available for anyone and everyone to take online. People get really excited about it because, well, first of all, it's from Harvard, which is kind of a big deal. But secondly, it's a really good course for anybody that's trying to learn about introduction to computer science and doesn't have any exposure into the field. And to be fair, it's really well organized and really well taught. The lecturer that teaches the course is really engaging. And I think they do a reall

Choose a summarization model (0 for 'Extractive summary', 1 for 'Abstractive summary'): 1


UnboundLocalError: local variable 'extractive_summary' referenced before assignment